In [1]:
# necessary imports
import tweepy as tweepy
from neo4j.v1 import GraphDatabase, basic_auth

In [2]:
# authenticaing twitter api user
consumer_key = ''
consumer_secret = ''
access_token =''
access_token_secret = ''
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [3]:
#retriving a tweet from home page
public_tweets = api.home_timeline()
public_tweets[0].text

'RT @PaytmTickets: Get ready for a smashing #Holi with rain dance, Punjabi Dhol, endless fun, food &amp; colours. Be a part of Rang Munch: https…'

### Twitter API rate limits

In [4]:
### checking rate limit - friends list
limit = api.rate_limit_status()
limit['resources']['friends']['/friends/list']['remaining']
limit['resources']['friends']['/friends/list']

{'limit': 15, 'remaining': 15, 'reset': 1488092352}

#### Calculating remaining time (in mins) before api limits reset
I find it helpful to check how many more minutes I have to wait before trying again

In [5]:
import datetime as dt
given_date =dt.datetime.fromtimestamp(
        int(limit['resources']['friends']['/friends/list']['reset'])
    ).strftime('%Y-%m-%d %H:%M:%S')
difference =  dt.datetime.strptime(given_date, "%Y-%m-%d %H:%M:%S")-dt.datetime.today()
min=difference.seconds/60
print ("Minute(s) remaining: ",min)

Minute(s) remaining:  15.083333333333334


### Key function(s) of tweepy wrapper
These fields are are stored in Neo4j (as properties) against a Person node.

In [6]:
# getting user details of a Person aka Narendra Modi
suser = api.get_user('narendramodi') # this returns a User model
print (suser.screen_name)
print (suser.followers_count)
print (suser.friends_count)
print(suser.location)
print(suser.created_at)

narendramodi
27398466
1638
India
2009-01-10 17:18:56


In [7]:
# get Narendra Modi's friends i.e. whome NM is following
friends = api.friends('narendramodi')

In [8]:
# defining a driver
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "welcome123"))
# initiating a server
session = driver.session()

#deleting ALL existing records
session.run("MATCH (n) DETACH DELETE n")

#adding a unique constraint; this ensures that same Person is not added twice
session.run("CREATE CONSTRAINT ON (a:Person) ASSERT a.screen_name IS UNIQUE")

session.close()

In [9]:
# defining a driver
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "welcome123"))
session = driver.session()

#adding narnendra modi details
label =suser.screen_name
session.run("CREATE (label:Person {screen_name: {screen_name}, name: {name}, followers_count: {followers_count}, friends_count:{friends_count},location:{location}})",
              {"name": suser.name, "screen_name": suser.screen_name, "followers_count":suser.followers_count,"friends_count": suser.friends_count,"location":suser.location }
               )
session.close()

In [10]:
# defining a driver
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "welcome123"))
session = driver.session()

i =0


#adding all persons - whom narendra modi is following
for user in friends:
    #checking if the user already exists
    output = session.run("MATCH (a:Person) WHERE a.screen_name ={check_name} return a.screen_name",
                             {"check_name":user.screen_name})
    exists =''
    for exists in output:
            True
    label =user.screen_name
    #adding the user if doesnt exist
    if exists=='':
        session.run("CREATE (label:Person {screen_name: {screen_name}, name: {name}, followers_count: {followers_count}, friends_count:{friends_count},location:{location}})",
              {"name": user.name, "screen_name": user.screen_name, "followers_count":user.followers_count,"friends_count": user.friends_count,"location":user.location }
               )
    else:
            print('Person already exists: ',user.screen_name)
            
    session.run("MATCH (a:Person),(b:Person) WHERE a.screen_name = {a_screen_name} AND b.screen_name = {b_screen_name} CREATE (a)-[r:FOLLOWING]->(b)",
               {"a_screen_name":suser.screen_name, "b_screen_name":user.screen_name })
    
    #temporary; ending the loop after 6 runs - dont exhaust twitter api limits
    i=i+1
    if i==6:
        break
    
    #finding friends of Modi's friends
    print ('sub_friends: '+ user.screen_name) #debugging why rate limits are getting exhausted
    sub_friends = api.friends(user.screen_name)
    #adding the friends of friends of Modi
    for sub_user in sub_friends:
        #checking if the sub_user already exists
        output = session.run("MATCH (a:Person) WHERE a.screen_name ={check_name} return a.screen_name",
                             {"check_name":sub_user.screen_name})
        exists =''
        for exists in output:
            True
        label =sub_user.screen_name
        #adding the user if doesnt exist
        if exists=='':
            
            session.run("CREATE (label:Person {screen_name: {screen_name}, name: {name}, followers_count: {followers_count}, friends_count:{friends_count},location:{location}})",
              {"name": sub_user.name, "screen_name": sub_user.screen_name, "followers_count":sub_user.followers_count,"friends_count": sub_user.friends_count,"location":sub_user.location }
               )
        else:
            print('Person already exists: ',sub_user.screen_name)
        session.run("MATCH (a:Person),(b:Person) WHERE a.screen_name = {a_screen_name} AND b.screen_name = {b_screen_name} CREATE (a)-[r:FOLLOWING]->(b)",
               {"a_screen_name":user.screen_name, "b_screen_name":sub_user.screen_name })
        

session.close()

sub_friends: blind_cricket
sub_friends: antoniocostapm
Person already exists:  narendramodi
sub_friends: DuttYogi
Person already exists:  klrahul11
sub_friends: klrahul11
Person already exists:  anilkumble1074
Person already exists:  SDhawan25
Person already exists:  BCCI
Person already exists:  karun126
sub_friends: karun126
Person already exists:  narendramodi
Person already exists:  coach_rsridhar
Person already exists:  VVSLaxman281
Person already exists:  anilkumble1074
Person already exists:  cheteshwar1
